In [ ]:
import numpy as np
import numpy.linalg as lin
import matplotlib.pyplot as plt
import torch

In [ ]:
from google.colab import drive # Link your drive if you are a colab user
drive.mount('/content/drive') # Models in this HW take a long time to get trained and make sure to save it her

Mounted at /content/drive


In [ ]:
data = np.load('/content/drive/MyDrive/IDLProject/massive_training_data_distances.pickle', allow_pickle = True)

In [ ]:
# (id) + (c1-> G, f, x, rho)  + (c2 -> G, f, x, rho) + (dist)

# Data Preparation

In [ ]:
c1_xs = []
c1_gs = []
c2_xs = []
c2_gs = []
dists = []

for i in range(len(data)):
  # if len(np.array(data[i]['c1']['G'])) == 99 and len(np.array(data[i]['c2']['G'])) == 99:
  c1_xs.append(np.array(data[i]['c1']['x']))
  # c1_gs.append(np.array(data[i]['c1']['G']))
  c2_xs.append(np.array(data[i]['c2']['x']))
  # c2_gs.append(np.array(data[i]['c2']['G']))
  dists.append(np.array(data[i]['dist']))


c1_xs = np.array(c1_xs)
# c1_gs = np.array(c1_gs)
c2_xs = np.array(c2_xs)
# c2_gs = np.array(c2_gs)
dists = np.array([dists]).T

In [ ]:
c1_xs.shape

(71649, 100, 2)

In [ ]:
c2_xs.shape

(71649, 100, 2)

In [ ]:
dists.shape

(71649, 1)

In [ ]:
cxy_data = np.concatenate((c1_xs,c2_xs),axis=2)

In [ ]:
label = dists

In [ ]:
cxy_data.shape

(71649, 100, 4)

In [ ]:
label.shape

(71649, 1)

# Data partition

In [ ]:
len_data = cxy_data.shape[0]
train_len = int(0.7*len_data)
val_len = int(0.1*len_data)
test_len =  len_data - train_len - val_len
generator = torch.Generator()
generator.manual_seed(0)

train_cxy_data, val_cxy_data, test_cxy_data = torch.utils.data.random_split(cxy_data, [train_len, val_len, test_len], generator=generator)
train_label, val_label, test_label = torch.utils.data.random_split(label, [train_len, val_len, test_len], generator=generator)

In [ ]:
print(len(train_cxy_data), len(train_label))
print(len(val_cxy_data), len(val_label))
print(len(test_cxy_data), len(test_label))

50154 50154
7164 7164
14331 14331


In [ ]:
train_cxy_data[1].shape

(100, 4)

# Dataloaders

In [ ]:
class ShapeTrainDataset(torch.utils.data.Dataset):
  def __init__(self, feature_data, label):
    self.feature_data = feature_data
    self.label = label
  def __len__(self):
    return len(self.feature_data)
  def __getitem__(self, ind):
    return self.feature_data[ind], self.label[ind]

In [ ]:
class ShapeTestDataset(torch.utils.data.Dataset):
  def __init__(self, feature_data):
    self.feature_data = feature_data
  def __len__(self):
    return len(self.feature_data)
  def __getitem__(self, ind):
    return self.feature_data[ind]

In [ ]:
# Create objects for the dataset class
train_data = ShapeTrainDataset(train_cxy_data, train_label) 
val_data = ShapeTrainDataset(val_cxy_data, val_label) 
test_data = ShapeTestDataset(test_cxy_data) 

In [ ]:
BATCH_SIZE = 64

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, num_workers= 10,batch_size=BATCH_SIZE, pin_memory= True,shuffle= True)
val_loader = torch.utils.data.DataLoader(val_data, num_workers= 10, batch_size=BATCH_SIZE, pin_memory= True,shuffle= False)
test_loader = torch.utils.data.DataLoader(test_data, num_workers= 2, batch_size=BATCH_SIZE, pin_memory= True,shuffle= False)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
print("Batch size: ", BATCH_SIZE)
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  64
Train dataset samples = 50154, batches = 784
Val dataset samples = 7164, batches = 112
Test dataset samples = 14331, batches = 224
